# Farcaster Insights

Created by https://warpcast.com/ghostlinkz.eth

Powered by https://neynar.com

Dashboard link: http://farcasterinsights.com

In [1]:
# Total Users

SELECT COUNT(DISTINCT fid) AS unique_fid_count
FROM profile_with_addresses;

---------------------------------------------------

# Total Casts 

select count(*) from casts

---------------------------------------------------

# Avg Casts Per Day / Last 30 Days

SELECT
    AVG(casts_per_day) AS average_casts_per_day
FROM (
    SELECT
        COUNT(*) AS casts_per_day
    FROM casts
    WHERE
        timestamp >= NOW() - INTERVAL '30 days'
    GROUP BY
        DATE(timestamp)
) AS daily_casts;

---------------------------------------------------

# Total casts per month

Select count(distinct(hash)), date_trunc('month', timestamp) as month from casts
group by 2
order by month desc
limit 10


---------------------------------------------------

# % of casts in channels / weekly

WITH RECURSIVE RecursiveReplyHierarchy AS (
    -- Base case: select all casts with "parent_url" and their children
    SELECT
        hash,
        timestamp,
        parent_hash,
        parent_url
    FROM
        casts
    WHERE
        parent_url IS NOT NULL

    UNION ALL

    -- Recursive case: select children of the previously selected rows
    SELECT
        c.hash,
        c.timestamp,
        c.parent_hash,
        c.parent_url
    FROM
        casts c
    INNER JOIN
        RecursiveReplyHierarchy r
    ON
        c.parent_hash = r.hash
)

SELECT
    -- Weekly total casts
    DATE_TRUNC('week', 
        CASE 
            WHEN c.timestamp < '2023-07-03' THEN '2023-07-03' 
            ELSE c.timestamp 
        END
    ) AS week_start,
    COUNT(*) AS total_casts,

    -- Total casts with "parent_url" for each week, including replies and nested replies
    SUM(CASE WHEN DATE_TRUNC('week', r.timestamp) = DATE_TRUNC('week', c.timestamp) THEN 1 ELSE 0 END) AS total_casts_with_parent_url,

    -- Calculate the weekly percentage of casts with "parent_url"
    (SUM(CASE WHEN DATE_TRUNC('week', r.timestamp) = DATE_TRUNC('week', c.timestamp) THEN 1 ELSE 0 END)::DECIMAL / COUNT(*)) * 100 AS percentage_casts_with_parent_url
FROM
    casts c
LEFT JOIN
    RecursiveReplyHierarchy r
ON
    c.hash = r.hash
WHERE
    c.timestamp >= '2023-07-03'
GROUP BY
    week_start
ORDER BY
    week_start;
    

---------------------------------------------------

# Monthly Active Users

SELECT
    COUNT(DISTINCT fid) AS monthly_active_users,
    date_trunc('month', "timestamp") AS month
FROM casts
WHERE "timestamp" >= NOW() - INTERVAL '9 months'
GROUP BY month
ORDER BY month DESC;

---------------------------------------------------

# Weekly Active Users

SELECT
    date_trunc('week', "timestamp") AS week,
    COUNT(DISTINCT fid) AS unique_users
FROM casts
WHERE "timestamp" >= NOW() - INTERVAL '24 weeks'
GROUP BY week
ORDER BY week;

---------------------------------------------------

# Distribution of Users by Cast Activity

SELECT
    'Group 1: 0 casts' AS cast_group,
    (SELECT COUNT(*) FROM profile_with_addresses) - (SELECT COUNT(DISTINCT fid) FROM casts) AS fid_count

UNION ALL

SELECT
    'Group 2: 1-10 casts' AS cast_group,
    COUNT(*) AS fid_count
FROM (
    SELECT DISTINCT fid
    FROM casts
    WHERE fid IN (
        SELECT fid
        FROM casts
        GROUP BY fid
        HAVING COUNT(*) BETWEEN 1 AND 10
    )
) AS group_2

UNION ALL

SELECT
    'Group 3: 11-50 casts' AS cast_group,
    COUNT(*) AS fid_count
FROM (
    SELECT DISTINCT fid
    FROM casts
    WHERE fid IN (
        SELECT fid
        FROM casts
        GROUP BY fid
        HAVING COUNT(*) BETWEEN 11 AND 50
    )
) AS group_3

UNION ALL

SELECT
    'Group 4: 51-100 casts' AS cast_group,
    COUNT(*) AS fid_count
FROM (
    SELECT DISTINCT fid
    FROM casts
    WHERE fid IN (
        SELECT fid
        FROM casts
        GROUP BY fid
        HAVING COUNT(*) BETWEEN 51 AND 100
    )
) AS group_4

UNION ALL

SELECT
    'Group 5: >100 casts' AS cast_group,
    COUNT(*) AS fid_count
FROM (
    SELECT DISTINCT fid
    FROM casts
    WHERE fid IN (
        SELECT fid
        FROM casts
        GROUP BY fid
        HAVING COUNT(*) > 100
    )
) AS group_5;


---------------------------------------------------

# Hourly Cast Activity

SELECT
    EXTRACT(HOUR FROM timestamp) AS hour_of_day,
    COUNT(*) AS casts_count
FROM
    casts
GROUP BY
    hour_of_day
ORDER BY
    hour_of_day;


---------------------------------------------------

# Top Channels / Last 24 hrs

WITH RECURSIVE ReplyHierarchy AS (
  SELECT
    hash,
    parent_url,
    parent_url AS original_parent_url,
    1 AS level,
    timestamp
  FROM
    casts
  WHERE
    parent_url IS NOT NULL
    AND timestamp >= (NOW() - INTERVAL '24 hours')

  UNION ALL

  SELECT
    c.hash,
    c.parent_url,
    rh.original_parent_url,
    rh.level + 1,
    c.timestamp
  FROM
    casts c
    INNER JOIN ReplyHierarchy rh ON c.parent_hash = rh.hash
),
RankedReplyHierarchy AS (
  SELECT
    ROW_NUMBER() OVER (ORDER BY COUNT(DISTINCT rh.hash) DESC) AS channel_rank,
    COUNT(DISTINCT rh.hash) AS count,
    CASE
      WHEN rh.original_parent_url LIKE 'chain://eip155:1/erc721:%' THEN
        CONCAT('https://zora.co/collect/eth:', SUBSTRING(rh.original_parent_url, 25))
      WHEN rh.original_parent_url LIKE 'chain://eip155:7777777/erc721:%' THEN
        CONCAT('https://zora.co/collect/zora:', SUBSTRING(rh.original_parent_url, 31))
      WHEN rh.original_parent_url LIKE 'chain://eip155:8453/erc721:%' THEN
        CONCAT('https://zora.co/collect/base:', SUBSTRING(rh.original_parent_url, 28))
      WHEN rh.original_parent_url LIKE 'chain://eip155:8453/erc1155:%' THEN
        CONCAT('https://zora.co/collect/base:', SUBSTRING(rh.original_parent_url, 29))
      ELSE
        rh.original_parent_url
    END AS parent_url
  FROM
    ReplyHierarchy rh
  GROUP BY
    rh.original_parent_url
)
SELECT
  * 
FROM
  RankedReplyHierarchy
LIMIT
  1000;
    

---------------------------------------------------

# Top Channels / All-Time


WITH RECURSIVE ReplyHierarchy AS (
  SELECT hash, parent_url, parent_url AS original_parent_url, 1 AS level
  FROM casts
  WHERE parent_url IS NOT NULL

  UNION ALL

  SELECT c.hash, c.parent_url, rh.original_parent_url, rh.level + 1
  FROM casts c
  INNER JOIN ReplyHierarchy rh ON c.parent_hash = rh.hash
),
RankedReplyHierarchy AS (
  SELECT 
    ROW_NUMBER() OVER (ORDER BY COUNT(DISTINCT rh.hash) DESC) AS channel_rank,
    COUNT(DISTINCT rh.hash) AS count, 
    CASE 
      WHEN rh.original_parent_url LIKE 'chain://eip155:1/erc721:%' THEN 
        CONCAT('https://zora.co/collect/eth:', SUBSTRING(rh.original_parent_url, 25))
      WHEN rh.original_parent_url LIKE 'chain://eip155:7777777/erc721:%' THEN 
        CONCAT('https://zora.co/collect/zora:', SUBSTRING(rh.original_parent_url, 31)) 
      WHEN rh.original_parent_url LIKE 'chain://eip155:8453/erc721:%' THEN 
        CONCAT('https://zora.co/collect/base:', SUBSTRING(rh.original_parent_url, 28))
      WHEN rh.original_parent_url LIKE 'chain://eip155:8453/erc1155:%' THEN 
        CONCAT('https://zora.co/collect/base:', SUBSTRING(rh.original_parent_url, 29))
      ELSE rh.original_parent_url
    END AS parent_url
  FROM ReplyHierarchy rh
  GROUP BY rh.original_parent_url
)
SELECT * FROM RankedReplyHierarchy
LIMIT 1000;


---------------------------------------------------

# Top 30 casts / likes / 24hrs utc

SELECT
    CONCAT('https://warpcast.com/', u.value, '/0x', LEFT(cast(encode(c.hash, 'hex') as text), 6)) AS cast_url_raw,
    c.text AS cast_text,
    c.fid AS user_fid,
    u.value AS username,
    c.timestamp AS cast_timestamp,
    COUNT(r.target_hash) AS reaction_count
FROM
    casts AS c
JOIN
    user_data AS u
    ON c.fid = u.fid
    AND u.type = '6'  -- value for usernames
LEFT JOIN
    reactions AS r
    ON c.hash = r.target_hash
    AND r.reaction_type = 1 -- value for recasts
WHERE
    c.timestamp >= NOW() - INTERVAL '24' HOUR
GROUP BY
    cast_url_raw, c.text, c.fid, u.value, cast_timestamp
ORDER BY
    reaction_count DESC
LIMIT
    30;


---------------------------------------------------

# Top 30 casts / replies / 24hrs utc

SELECT
    CONCAT('https://warpcast.com/', u.value, '/0x', LEFT(cast(encode(c.hash, 'hex') as text), 6)) AS cast_url_raw,
    c.text AS cast_text,
    c.fid AS user_fid,
    u.value AS username,
    c.timestamp AS cast_timestamp,
    COUNT(r.hash) AS reply_count
FROM
    casts AS c
JOIN
    user_data AS u
    ON c.fid = u.fid
    AND u.type = '6'  -- value for usernames
LEFT JOIN
    casts AS r
    ON c.hash = r.parent_hash
WHERE
    c.timestamp >= NOW() - INTERVAL '24' HOUR
GROUP BY
    cast_url_raw, c.text, c.fid, u.value, cast_timestamp
ORDER BY
    reply_count DESC
LIMIT
    30;
    
---------------------------------------------------

# Top 30 casts / recasts / 24hrs utc

SELECT
    CONCAT('https://warpcast.com/', u.value, '/0x', LEFT(cast(encode(c.hash, 'hex') as text), 6)) AS cast_url_raw,
    c.text AS cast_text,
    c.fid AS user_fid,
    u.value AS username,
    c.timestamp AS cast_timestamp,
    COUNT(r.target_hash) AS reaction_count
FROM
    casts AS c
JOIN
    user_data AS u
    ON c.fid = u.fid
    AND u.type = '6'  -- value for usernames
LEFT JOIN
    reactions AS r
    ON c.hash = r.target_hash
    AND r.reaction_type = 2 -- value for recasts
WHERE
    c.timestamp >= NOW() - INTERVAL '24' HOUR
GROUP BY
    cast_url_raw, c.text, c.fid, u.value, cast_timestamp
ORDER BY
    reaction_count DESC
LIMIT
    30;

---------------------------------------------------

# Power Users: 100+ Casts (Last 4 Consecutive Months)

WITH MonthlyCastCounts AS (
  SELECT
    c.fid,
    DATE_TRUNC('month', c.timestamp) AS month,
    COUNT(*) AS cast_count
  FROM
    casts c
  WHERE
    c.timestamp >= NOW() - INTERVAL '4 months'  -- Filter for the last 6 months
  GROUP BY
    c.fid,
    month
),
DistinctMonths AS (
  SELECT DISTINCT month
  FROM MonthlyCastCounts
)

SELECT
  ROW_NUMBER() OVER (ORDER BY mc.total_casts DESC) AS ranking,
  u.fid AS user_fid,
  CONCAT('https://warpcast.com/', u.value) AS user_url,  -- Generate the user URL
  mc.total_casts
FROM (
  SELECT
    c.fid,
    COUNT(*) AS counted_months
  FROM
    MonthlyCastCounts c
  WHERE
    c.cast_count >= 100
  GROUP BY
    c.fid
  HAVING
    COUNT(*) = (SELECT COUNT(*) FROM DistinctMonths)
) AS UsersWithCastsEachMonth
INNER JOIN user_data u ON UsersWithCastsEachMonth.fid = u.fid
INNER JOIN (
  SELECT
    fid,
    SUM(cast_count) AS total_casts
  FROM
    MonthlyCastCounts
  GROUP BY
    fid
) AS mc ON u.fid = mc.fid
WHERE
  u.type = 6
ORDER BY
  mc.total_casts DESC;
    

---------------------------------------------------

# Top 50 Most Mentioned Users / All-Time

WITH MentionCounts AS (
  SELECT
    UNNEST(mentions) AS mentioned_fid,
    COUNT(*) AS mention_count
  FROM
    casts
  WHERE
    mentions IS NOT NULL
  GROUP BY
    mentioned_fid
),
TopMentionedUsers AS (
  SELECT
    mc.mentioned_fid,
    mc.mention_count,
    'https://warpcast.com/' || u.value AS user_url  -- Generate user URLs
  FROM
    MentionCounts mc
  INNER JOIN
    user_data AS u
    ON mc.mentioned_fid = u.fid
    AND u.type = '6'  -- Value for usernames
  ORDER BY
    mc.mention_count DESC
  LIMIT 50  -- Change this limit to get more or fewer top mentioned users
)

SELECT
  ROW_NUMBER() OVER (ORDER BY mention_count DESC) AS ranking,
  tmu.mentioned_fid AS fid,
  tmu.user_url AS username,  -- Use the generated user URL
  tmu.mention_count
FROM
  TopMentionedUsers tmu
ORDER BY
  mention_count DESC;
